union de archivo excek de acotur_huila_competitiva_CTravel

In [7]:
import pandas as pd
import glob

# Carpeta donde tienes los archivos Excel
ruta = "../acotur_huila_competitiva_CTravel\output\excel\portuColombia_final.xlsx"

# Diccionario para estandarizar columnas
mapa_columnas = {
    "fuente": "Fuente",
    "Fuente": "Fuente",
    "nombre": "Nombre",
    "Nombre": "Nombre",
    "municipio": "Municipio",
    "Municipio": "Municipio",
    "departamento": "Departamento",
    "Departamento": "Departamento",
    "rnt": "RNT",
    "RNT": "RNT",
    "descripcion": "Descripción",
    "descripción": "Descripción",
    "Descripción": "Descripción",
    "categorias": "Categorías",
    "categorías": "Categorías",
    "categoria": "Categorías",
    "Categorías": "Categorías",
    "subtipo": "Subtipo",
    "certificacion": "Certificaciones",
    "Certificaciones": "Certificaciones",
    "email": "Email",
    "Email": "Email",
    "redes_sociales": "Redes Sociales",
    "Redes Sociales": "Redes Sociales",
    "redes sociales": "Redes Sociales",
    "url": "URL",
    "URL": "URL",
    "decreto": "Decreto",
    "detalle": "Detalle",
    "dirección": "Dirección",
    "teléfonos": "Teléfonos",
    "imágenes": "Imágenes"
}

# Lista para guardar DataFrames
df_list = []
total_registros = 0

for archivo in glob.glob(ruta):
    # Leemos todas las hojas
    hojas = pd.read_excel(archivo, sheet_name=None)
    
    for nombre, df in hojas.items():
        registros_antes = len(df)
        total_registros += registros_antes
        
        print(f"Archivo: {archivo.split('/')[-1]} | Hoja: {nombre} | Registros: {registros_antes}")
        
        # Normalizar nombres de columnas (minúsculas y sin espacios)
        df = df.rename(columns=lambda x: str(x).strip().lower())
        # Reemplazar según el diccionario
        df = df.rename(columns=mapa_columnas)
        
        # Agregar columnas de origen
        df["Fuente_Archivo"] = archivo.split("/")[-1]
        df["Fuente_Hoja"] = nombre
        
        df_list.append(df)

# Concatenar todo
df_final = pd.concat(df_list, ignore_index=True)

# Reemplazar valores en blanco o NaN en la columna Fuente
df_final["Fuente"] = df_final["Fuente"].fillna("Acotur")
df_final.loc[df_final["Fuente"].astype(str).str.strip() == "", "Fuente"] = "Acotur"

print("\n----------------------------------")
print(f"🔹 Total registros antes de unir: {total_registros}")
print(f"🔹 Total registros después de unir: {len(df_final)}")
print("----------------------------------")

# Exportar
df_final.to_excel("dataset_unificado.xlsx", index=False)

Archivo: acotur_huila_competitiva_CTravel\output\excel\portuColombia_final.xlsx | Hoja: Caquetá | Registros: 82
Archivo: acotur_huila_competitiva_CTravel\output\excel\portuColombia_final.xlsx | Hoja: Huila | Registros: 97
Archivo: acotur_huila_competitiva_CTravel\output\excel\portuColombia_final.xlsx | Hoja: Putumayo | Registros: 3
Archivo: acotur_huila_competitiva_CTravel\output\excel\portuColombia_final.xlsx | Hoja: Tolima | Registros: 4

----------------------------------
🔹 Total registros antes de unir: 186
🔹 Total registros después de unir: 186
----------------------------------


In [14]:
import pandas as pd

# -----------------------------
# Rutas de archivos
# -----------------------------
archivo_excel = "dataset_unificado.xlsx"   # el que generaste antes (Acotur + Caquetá Travel)
archivo_csv = "..\Compilado\consolidado.csv"         # el CSV nuevo que pasaste

# -----------------------------
# Leer archivos
# -----------------------------
df_excel = pd.read_excel(archivo_excel)
df_csv = pd.read_csv(archivo_csv)

print(f"📊 Registros iniciales Excel: {len(df_excel)}")
print(f"📊 Registros iniciales CSV: {len(df_csv)}")

# -----------------------------
# Normalizar columnas para unir
# -----------------------------
# Columnas finales SIN duplicar Titulo / Categoria
columnas_finales = [
    "Fuente","Nombre","Municipio","RNT","Descripción","Categorías","Certificaciones",
    "Email","Redes Sociales","URL","Departamento","Detalle","Dirección","Teléfonos",
    "Imágenes","Fuente_Archivo","Fuente_Hoja","Decreto",
    "Tipo","Fecha_Extraccion","Precio"
]


# --- Transformar df_excel ---
df_excel["Tipo"] = None
df_excel["Fecha_Extraccion"] = None
df_excel["Precio"] = None

# --- Transformar df_csv ---
df_csv_ren = pd.DataFrame()
df_csv_ren["Fuente"] = df_csv["fuente"]
df_csv_ren["Nombre"] = df_csv["titulo"]
df_csv_ren["Municipio"] = df_csv["ubicacion"]
df_csv_ren["RNT"] = None
df_csv_ren["Descripción"] = df_csv["descripcion"]
df_csv_ren["Categorías"] = df_csv["categoria"]
df_csv_ren["Certificaciones"] = None
df_csv_ren["Email"] = None
df_csv_ren["Redes Sociales"] = None
df_csv_ren["URL"] = df_csv["enlace"]
df_csv_ren["Departamento"] = df_csv["ubicacion"]
df_csv_ren["Detalle"] = df_csv["detalles"]
df_csv_ren["Dirección"] = None
df_csv_ren["Teléfonos"] = df_csv["telefono"]
df_csv_ren["Imágenes"] = df_csv["imagen"]
df_csv_ren["Fuente_Archivo"] = df_csv["origen_archivo"]
df_csv_ren["Fuente_Hoja"] = None
df_csv_ren["Decreto"] = None
df_csv_ren["Tipo"] = df_csv["tipo"]
df_csv_ren["Fecha_Extraccion"] = df_csv["fecha_extraccion"]
df_csv_ren["Precio"] = df_csv["precio"]

# Asegurar el mismo orden de columnas
df_excel = df_excel.reindex(columns=columnas_finales)
df_csv_ren = df_csv_ren.reindex(columns=columnas_finales)

# -----------------------------
# Unir datasets
# -----------------------------
df_final = pd.concat([df_excel, df_csv_ren], ignore_index=True)

print("\n✅ Unión completada")
print(f"📊 Registros finales: {len(df_final)}")
print(f"🔹 Registros con Fuente='Acotur': {(df_final['Fuente'] == 'Acotur').sum()}")

# -----------------------------
# Exportar resultado
# -----------------------------
df_final.to_excel("dataset_final.xlsx", index=False)
print("📁 Archivo exportado como dataset_final.xlsx")


📊 Registros iniciales Excel: 186
📊 Registros iniciales CSV: 302

✅ Unión completada
📊 Registros finales: 488
🔹 Registros con Fuente='Acotur': 12
📁 Archivo exportado como dataset_final.xlsx
